## Web scrapping de IMDB

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd


In [60]:
URL = 'https://www.imdb.com'
r = requests.get(URL + '/chart/top')
soup = bs(r.text, 'lxml')
movies_grid = soup.find_all(class_="titleColumn")

movies_dict = {'movie_title': [],
                'year': [],
                'place': [],
                'star_cast': [],
                'rating': [],
                'vote': [],
                'link': [],
                }                

count = 0
print(f'En busca de películas:')
for movie in movies_grid:
    count += 1
    url_addition = movie.find('a')['href']
    r = requests.get(URL + url_addition)
    soup_movie = bs(r.text, 'lxml')
    movies_dict['place'].append(count)
    movies_dict['link'].append(url_addition)
    print(f'---- Posición {count} (url: {url_addition}) ----')
    try:
        movie_title = soup_movie.find('h1').text
        movies_dict['movie_title'].append(movie_title)
        print(f'Título: {movie_title}')
    except:
        movies_dict['movie_title'].append(None)
        print('Título: (sin título)')
    try:
        year = soup_movie.find(class_='ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh').text
        movies_dict['year'].append(year)
        print(f'Año: {year}')
    except:
        movies_dict['year'].append(None)
        print(f'Año: (sin año)')
    try:
        star_cast = []
        full_cast = soup_movie.find(class_='ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt')
        for element in full_cast:
            if 'Director' in str(element.find(class_='ipc-metadata-list-item__label ipc-metadata-list-item__label--btn')):
                director = element.find(class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link').text + ' (dir.)'
                star_cast.append(director)
            else:
                if 'Stars' in str(element.find(class_='ipc-metadata-list-item__label ipc-metadata-list-item__label--link')):
                    for star in element.find_all(class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'):
                        star_cast.append(star.text)
        star_cast = ', '.join(star_cast)
        movies_dict['star_cast'].append(star_cast)
        print(f'Estrellas: {star_cast}')
    except:
        movies_dict['star_cast'].append(None)
        print('Estrellas: (sin estrellas)')
    try:
        rating = soup_movie.find(class_='sc-7ab21ed2-1 jGRxWM').text
        movies_dict['rating'].append(rating)
        print(f'Rating: {rating}')
    except:
        movies_dict['rating'].append(None)
        print('Rating: (sin rating)')
    try:
        votos = soup_movie.find(class_='sc-7ab21ed2-3 dPVcnq').text
        movies_dict['vote'].append(votos)
        print(f'Votos: {votos}')
    except:
        movies_dict['vote'].append(None)
        print('Votos: (sin votos)')
    print('\n')

df_movies = pd.DataFrame(movies_dict)


En busca de películas:
---- Posición 1 (url: /title/tt0111161/) ----
Título: Cadena perpetua
Año: 1994
Estrellas: Frank Darabont (dir.), Tim Robbins, Morgan Freeman, Bob Gunton
Rating: 9.3
Votos: 2.7M


---- Posición 2 (url: /title/tt0068646/) ----
Título: El padrino
Año: 1972
Estrellas: Francis Ford Coppola (dir.), Marlon Brando, Al Pacino, James Caan
Rating: 9.2
Votos: 1.8M


---- Posición 3 (url: /title/tt0468569/) ----
Título: El caballero oscuro
Año: 2008
Estrellas: Christopher Nolan (dir.), Christian Bale, Heath Ledger, Aaron Eckhart
Rating: 9.0
Votos: 2.6M


---- Posición 4 (url: /title/tt0071562/) ----
Título: El padrino (parte II)
Año: 1974
Estrellas: Francis Ford Coppola (dir.), Al Pacino, Robert De Niro, Robert Duvall
Rating: 9.0
Votos: 1.3M


---- Posición 5 (url: /title/tt0050083/) ----
Título: 12 hombres sin piedad
Año: 1957
Estrellas: Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb, Martin Balsam
Rating: 9.0
Votos: 787K


---- Posición 6 (url: /title/tt0108052/) ----
Títul

In [61]:
df_movies


,movie_title,year,place,star_cast,rating,vote,link
0,Cadena perpetua,1994,1,"Frank Darabont (dir.), Tim Robbins, Morgan Fre...",9.3,2.7M,/title/tt0111161/
1,El padrino,1972,2,"Francis Ford Coppola (dir.), Marlon Brando, Al...",9.2,1.8M,/title/tt0068646/
2,El caballero oscuro,2008,3,"Christopher Nolan (dir.), Christian Bale, Heat...",9.0,2.6M,/title/tt0468569/
3,El padrino (parte II),1974,4,"Francis Ford Coppola (dir.), Al Pacino, Robert...",9.0,1.3M,/title/tt0071562/
4,12 hombres sin piedad,1957,5,"Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb,...",9.0,787K,/title/tt0050083/
...,...,...,...,...,...,...,...
245,Dersu Uzala (El cazador),1975,246,"Akira Kurosawa (dir.), Maksim Munzuk, Yuriy So...",8.2,30K,/title/tt0071411/
246,Aladdín,1992,247,"Ron Clements (dir.), Scott Weinger, Robin Will...",8.0,420K,/title/tt0103639/
247,Criadas y señoras,2011,248,"Tate Taylor (dir.), Viola Davis, Emma Stone, O...",8.1,460K,/title/tt1454029/
248,El gigante de hierro,1999,249,"Brad Bird (dir.), Eli Marienthal, Harry Connic...",8.1,199K,/title/tt0129167/
